In [4]:
%load_ext Cython

In [12]:
%%cython
import numpy as np
cimport numpy as np

cdef class TAHelper:
    cpdef list indicators_name
    
    cdef int i
    cdef int length
    cdef np.ndarray high
    cdef np.ndarray low
    cdef np.ndarray close
    cdef np.ndarray vol
    
    # 计算DX需要的变量
    cdef int dx_n
    cdef np.ndarray deltahigh
    cdef np.ndarray deltalow
    cdef np.ndarray plusdm
    cdef np.ndarray minusdm
    cdef np.ndarray admplus
    cdef np.ndarray admminus
    cdef np.ndarray tr
    cdef np.ndarray dx_atr
    cdef np.ndarray plusdi
    cdef np.ndarray minusdi
    cdef np.ndarray dx
    
    # 计算ADX需要的变量
    cdef np.ndarray adx
    
    # 计算MFI需要的变量
    cdef int mfi_n
    cdef np.ndarray typical_price
    cdef np.ndarray money_flow
    cdef np.ndarray positive_money_flow
    cdef np.ndarray negative_money_flow
    cdef np.ndarray money_ratio
    cdef np.ndarray mfi
    
    # 计算APO需要的变量
    cdef int apo_fast_n
    cdef int apo_slow_n
    cdef np.ndarray apo_fastma 
    cdef np.ndarray apo_slowma
    cdef np.ndarray apo

    # 计算MACD需要的变量
    cdef int macd_fast_n
    cdef int macd_slow_n
    cdef int macd_signal_n
    cdef np.ndarray macd_fastema 
    cdef np.ndarray macd_slowema
    cdef np.ndarray macd
    cdef np.ndarray macdsignal
    cdef np.ndarray macdhist
    
    # 计算NATR需要的变量
    cdef int natr_n
    cdef np.ndarray natr_atr
    cdef np.ndarray natr
    
    # 计算PLUS_DM需要的变量
    cdef int plus_dm_n
    cdef np.ndarray plus_dm
    cdef double init_plusdm
    
    # 计算PLUS_DI需要的变量
    cdef int plus_di_n
    cdef np.ndarray true_range_n
    cdef np.ndarray plus_di
    cdef double init_true_range_n
    
    # 计算RSI需要的变量
    cdef int rsi_n
    cdef np.ndarray upavg
    cdef np.ndarray dnavg
    cdef np.ndarray rsi
    
    # 计算TEMA需要的变量
    cdef int tema_n
    cdef np.ndarray tema_ema1
    cdef np.ndarray tema_ema2
    cdef np.ndarray tema_ema3
    cdef np.ndarray tema
    
    # 计算自定义指标1需要的变量
    cdef int customized_indicator_n
    cdef np.ndarray customized_indicator
    cdef np.ndarray customized_indicator_max_part
    cdef np.ndarray customized_indicator_abs_part
    cdef np.ndarray customized_indicator_sma_max_part
    cdef np.ndarray customized_indicator_sma_abs_part
    
    # 计算CCI需要的变量
    cdef int cci_n
    cdef np.ndarray avg_typical_price
    cdef np.ndarray mean_deviation
    cdef np.ndarray cci
    
    # 计算KDJ需要的变量
    cdef int kdj_n
    cdef int k_n
    cdef int d_n
    cdef np.ndarray highest_high
    cdef np.ndarray lowest_low
    cdef np.ndarray rsv
    cdef np.ndarray k
    cdef np.ndarray d
    cdef np.ndarray j
    
    def __init__(self, mom_window=12, std_window=26):
        assert mom_window*2 < std_window
        self.indicators_name = ['DX', 'ADX', 'MFI', 'APO', 'MACD', 'MACDsignal', 'MACDhist', 'NATR', 'PLUS_DM', 'RSI', 'TEMA', 'CUSTOMIZED_INDICATOR', 'CCI', 'RSV', 'KDJ_K', 'KDJ_D', 'KDJ_J', 'ATR', 'PLUS_DI']
        self.i = -1
        self.length = 10000
        self.high = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.low = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.close = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.vol = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算DX需要的变量
        self.dx_n = mom_window
        self.deltahigh = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.deltalow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.plusdm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.minusdm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.admplus = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.admminus = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dx_atr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.plusdi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.minusdi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dx = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算ADX需要的变量
        self.adx = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算MFI需要的变量
        self.mfi_n = mom_window
        self.typical_price = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.positive_money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.negative_money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.money_ratio = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.mfi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算APO需要的变量
        self.apo_fast_n = mom_window
        self.apo_slow_n = std_window
        self.apo_fastma = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.apo_slowma = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.apo = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算MACD需要的变量
        self.macd_fast_n = mom_window
        self.macd_slow_n = std_window
        self.macd_signal_n = round(0.75 * mom_window)
        self.macd_fastema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macd_slowema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macd = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macdsignal = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macdhist = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算NATR需要的变量
        self.natr_n = mom_window
        self.natr_atr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.natr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算PLUS_DM需要的变量
        self.plus_dm_n = mom_window
        self.plus_dm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.init_plusdm = 0
        
        # 计算PLUS_DI需要的变量
        self.plus_di_n = self.plus_dm_n
        self.true_range_n = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.plus_di = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.init_true_range_n = 0
        
        # 计算RSI需要的变量
        self.rsi_n = mom_window
        self.upavg = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dnavg = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.rsi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算TEMA需要的变量
        self.tema_n = std_window
        self.tema_ema1 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema_ema2 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema_ema3 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算自定义指标1需要的变量
        self.customized_indicator_n = std_window
        self.customized_indicator = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.customized_indicator_max_part = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.customized_indicator_abs_part = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.customized_indicator_sma_max_part = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.customized_indicator_sma_abs_part = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算CCI需要的变量
        self.cci_n = mom_window
        self.avg_typical_price = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.mean_deviation = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.cci = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算KDJ需要的变量
        self.kdj_n = mom_window
        self.k_n = round(0.75*mom_window)
        self.d_n = round(0.75*mom_window)
        self.highest_high = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.lowest_low = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.rsv = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.k = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.d = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.j = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
    cdef double TrueRange(self, double high, double low, double preClose):
        return max([high-low, abs(high-preClose), abs(preClose-low)])
    
    cdef double DX_inc(self, double plusDI, double minusDI):
        return 100 * abs(plusDI - minusDI) / (plusDI + minusDI + np.finfo(float).eps)
    
    cdef double MFI_inc(self, double MoneyRatio):
        return 100 - (100/(1+MoneyRatio))
    
    cdef double APO_inc(self, double fastMA, double slowMA):    
        return fastMA - slowMA
    
    cdef np.ndarray MACD_inc(self, double fastema, double slowema, double preSignal, int signal_n):
        cdef double macd = fastema - slowema
        cdef double signal = ((signal_n-1)/(signal_n+1))*preSignal + (2/(signal_n+1))*macd
        cdef double hist = macd - signal
        return np.array([macd, signal, hist])
    
    cdef double NATR_inc(self, double close, double atr):
        cdef double natr = 100*atr/close
        return natr
    
    cdef np.ndarray dm_one_period(self, double high, double prehigh, double low, double prelow):
        cdef double deltahigh = high - prehigh
        cdef double deltalow = prelow - low
        cdef double plusdm
        cdef double minusdm

        if (deltahigh < 0 and deltalow < 0) or deltahigh == deltalow:
            plusdm = 0
            minusdm = 0
        elif deltahigh > deltalow:
            plusdm = deltahigh
            minusdm = 0
        else:
            plusdm = 0
            minusdm = deltalow

        return np.array([plusdm, minusdm])
    
    cdef double PLUS_DM_inc(self, double high, double prehigh, double low, double prelow, double pre_plus_dm_n, int n):
        plusdm, _ = self.dm_one_period(high, prehigh, low, prelow)
        plus_dm_n = pre_plus_dm_n - pre_plus_dm_n/n + plusdm

        return plus_dm_n
    
    cdef double PLUS_DI_inc(self, double plus_dm_n, double true_range_n):
        return plus_dm_n / true_range_n * 100
    
    cdef double true_range_n_inc(self, double pre_true_range_n, double preClose, double high, double low, int n):
        cdef double tr
        tr = self.TrueRange(high, low, preClose)
        true_range_n = pre_true_range_n - pre_true_range_n / n + tr
        return true_range_n  
    
    cdef double RSI_inc(self, double close, double preClose, double preUpavg, double preDnavg, int n=14):
        cdef double up
        cdef double dn
        if close > preClose:
            up = close - preClose
            dn = 0
        else:
            up = 0
            dn = preClose - close

        upavg = (preUpavg*(n-1) + up) / n
        dnavg = (preDnavg*(n-1) + dn) / n
        rsi = 100 * upavg/(upavg + dnavg)

        return rsi
    
    cdef double TEMA_inc(self, double ema1, double ema2, double ema3):
        return 3*ema1 - 3*ema2 + ema3
    
    cdef double customized_indicator_inc(self, double sma_max_part, double sma_abs_part):
        return sma_max_part/sma_abs_part*100
 
    cdef double CCI_inc(self, double typical_price, double avg_typical_price, double avg_deviation):
        return (typical_price - avg_typical_price)/(0.015*avg_deviation)
    
    cdef dx_depends_update(self):
        # DX 依赖变量更新
        # 更新 typicalprice, moneyflow
        self.typical_price[self.i] = (self.high[self.i] + self.low[self.i] + self.close[self.i]) / 3
        self.money_flow[self.i] = self.typical_price[self.i] * self.vol[self.i]
        
        if self.i == 0:
            pass
        else:
            self.deltahigh[self.i] = self.high[self.i] - self.high[self.i-1]
            self.deltalow[self.i] = self.low[self.i-1] - self.low[self.i]
            self.tr[self.i] = self.TrueRange(self.high[self.i], self.low[self.i], self.close[self.i-1])
            # 更新 plusdm minusdm
            if (self.deltahigh[self.i] < 0 and self.deltalow[self.i] < 0) or \
            (self.deltahigh[self.i] == self.deltalow[self.i]):
                self.plusdm[self.i] = 0
                self.minusdm[self.i] = 0
            elif self.deltahigh[self.i] > self.deltalow[self.i]:
                self.plusdm[self.i] = self.deltahigh[self.i]
                self.minusdm[self.i] = 0
            elif self.deltahigh[self.i] < self.deltalow[self.i]:
                self.plusdm[self.i] = 0
                self.minusdm[self.i] = self.deltalow[self.i]
                
            if self.i == 1:
                # 更新第一个 adm
                
                self.admplus[self.i] = self.plusdm[self.i]
                self.admminus[self.i] = self.minusdm[self.i]
                
                # 更新第一个 atr
                self.dx_atr[self.i] = self.tr[self.i]  
            else:
                # 更新第二个及以后的 adm
                
                self.admplus[self.i] = (self.plusdm[self.i] + (self.dx_n-1)*self.admplus[self.i-1]) / self.dx_n
                self.admminus[self.i] = (self.minusdm[self.i] + (self.dx_n-1)*self.admminus[self.i-1]) / self.dx_n

                # 更新第二个及以后的 atr
                self.dx_atr[self.i] = (self.tr[self.i] + (self.dx_n-1)*self.dx_atr[self.i-1]) / self.dx_n
            # 更新 di
            self.plusdi[self.i] = 100 * self.admplus[self.i] / self.dx_atr[self.i]
            self.minusdi[self.i] = 100 * self.admminus[self.i] / self.dx_atr[self.i]
            # 更新 dx
            self.dx[self.i] = self.DX_inc(self.plusdi[self.i], self.minusdi[self.i])
            
            # 更新 adx
            if self.i == 1:
                self.adx[self.i] = self.dx[self.i]
            else:
                self.adx[self.i] = ((self.dx_n-1) * self.adx[self.i-1] + self.dx[self.i]) / self.dx_n
                
    cdef mfi_depends_update(self):
        # MFI 依赖变量更新
        # 更新 typicalprice, moneyflow
        self.typical_price[self.i] = (self.high[self.i] + self.low[self.i] + self.close[self.i]) / 3
        self.money_flow[self.i] = self.typical_price[self.i] * self.vol[self.i]

        if self.i == 0:
            pass
        else:
            # 更新 positivemoneyflow, negativemoneyflow
            if self.typical_price[self.i] > self.typical_price[self.i-1]:
                self.positive_money_flow[self.i] = self.money_flow[self.i]
                self.negative_money_flow[self.i] = 0
            else:
                self.positive_money_flow[self.i] = 0
                self.negative_money_flow[self.i] = self.money_flow[self.i]

            # 更新 moneyratio
            if self.i >= self.mfi_n:
                self.money_ratio[self.i] = ( self.positive_money_flow[self.i-self.mfi_n+1:self.i+1].sum() /                                                          self.negative_money_flow[self.i-self.mfi_n+1:self.i+1].sum() )
                # 更新 moneyflowindex
                self.mfi[self.i] = self.MFI_inc(self.money_ratio[self.i])
           
    cdef apo_depends_update(self):
        # APO 依赖变量更新
        # 更新fastma
        if self.i >= self.apo_fast_n-1:
            self.apo_fastma[self.i] = self.close[self.i-self.apo_fast_n+1:self.i+1].mean()
        # 更新slowma
        if self.i >= self.apo_slow_n-1:
            self.apo_slowma[self.i] = self.close[self.i-self.apo_slow_n+1:self.i+1].mean()
            # 更新apo
            self.apo[self.i] = self.APO_inc(self.apo_fastma[self.i], self.apo_slowma[self.i])
            
    cdef macd_depends_update(self):
        # MACD 依赖变量更新
        # 更新第一个fastema slowema
        if self.i == 0:
            self.macd_fastema[self.i] = self.close[self.i]
            self.macd_slowema[self.i] = self.close[self.i]
            self.macd[self.i] = self.macd_fastema[self.i] - self.macd_slowema[self.i]
            self.macdsignal[self.i] = self.macd[self.i]
            self.macdhist[self.i] = self.macd[self.i] - self.macdsignal[self.i]
        else:
            self.macd_fastema[self.i] = self.macd_fastema[self.i-1]*((self.macd_fast_n-1)/(self.macd_fast_n+1)) + (2/(self.macd_fast_n+1))*self.close[self.i]
            self.macd_slowema[self.i] = self.macd_slowema[self.i-1]*((self.macd_slow_n-1)/(self.macd_slow_n+1)) + (2/(self.macd_slow_n+1))*self.close[self.i]
            self.macd[self.i], self.macdsignal[self.i], self.macdhist[self.i] = self.MACD_inc(self.macd_fastema[self.i], self.macd_slowema[self.i], self.macdsignal[self.i-1], self.macd_signal_n)                
            
    cdef natr_depends_update(self):
        # NATR 依赖变量更新
        if self.i == 0:
            pass
        else:
            if self.i == 1:
                # 更新第一个natr_atr
                self.natr_atr[self.i] = self.tr[self.i] 
            elif self.i >= 2:
                self.natr_atr[self.i] = (self.tr[self.i] + (self.natr_n-1)*self.natr_atr[self.i-1]) / self.natr_n
                self.natr[self.i] = self.NATR_inc(self.close[self.i], self.natr_atr[self.i])
                
    cdef plus_dm_depends_update(self):
        # PLUS_DM 依赖变量更新
        if self.i == 0:
            pass
        elif self.i < self.plus_dm_n:
            self.init_plusdm += self.dm_one_period(self.high[self.i], self.high[self.i-1], self.low[self.i], self.low[self.i])[0]
            if self.i == self.plus_dm_n-1:
                self.plus_dm[self.i] = self.init_plusdm
        else:
            self.plus_dm[self.i] = self.PLUS_DM_inc(self.high[self.i], self.high[self.i-1], self.low[self.i], self.low[self.i-1], self.plus_dm[self.i-1], self.plus_dm_n) 
            
    cdef plus_di_depends_update(self):
        # PLUS_DI 依赖变量更新
        if self.i == 0:
            pass
        elif self.i < self.plus_di_n:
            self.init_true_range_n += self.TrueRange(self.high[self.i], self.low[self.i], self.close[self.i-1])
            if self.i == self.plus_di_n-1:
                self.true_range_n[self.i] = self.init_true_range_n
                self.plus_di[self.i] = self.PLUS_DI_inc(self.plus_dm[self.i], self.true_range_n[self.i])
        else:
            self.true_range_n[self.i] = self.true_range_n_inc(self.true_range_n[self.i-1], self.close[self.i-1], self.high[self.i], self.low[self.i], self.plus_di_n)
            self.plus_di[self.i] = self.PLUS_DI_inc(self.plus_dm[self.i], self.true_range_n[self.i])
    
    cdef rsi_depends_update(self):
        # RSI 依赖变量更新
        if self.i == 0:
            pass
        else:
            if self.close[self.i] > self.close[self.i-1]:
                up = self.close[self.i] - self.close[self.i-1]
                dn = 0
            else:
                up = 0
                dn = self.close[self.i-1] - self.close[self.i]
                
            if self.i == 1:
                self.upavg[self.i] = up
                self.dnavg[self.i] = dn
            else:
                self.rsi[self.i] = self.RSI_inc(self.close[self.i], self.close[self.i-1], self.upavg[self.i-1], self.dnavg[self.i-1], self.rsi_n)
                self.upavg[self.i] = ((self.rsi_n-1)*self.upavg[self.i-1] + up) / self.rsi_n
                self.dnavg[self.i] = ((self.rsi_n-1)*self.dnavg[self.i-1] + dn) / self.rsi_n    
 
    cdef tema_depends_update(self):
        # TEMA 依赖变量更新
        if self.i == 0:
            # 更新第一个ema1 ema2 ema3
            self.tema_ema1[self.i] = self.close[self.i]
            self.tema_ema2[self.i] = self.tema_ema1[self.i]
            self.tema_ema3[self.i] = self.tema_ema2[self.i]
        else:
            self.tema_ema1[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema1[self.i-1] + (2/(self.tema_n+1))*self.close[self.i]
            self.tema_ema2[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema2[self.i-1] + (2/(self.tema_n+1))*self.tema_ema1[self.i]
            self.tema_ema3[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema3[self.i-1] + (2/(self.tema_n+1))*self.tema_ema2[self.i]
            
        self.tema[self.i] = self.TEMA_inc(self.tema_ema1[self.i], self.tema_ema2[self.i], self.tema_ema3[self.i])  
        
    cdef customized_indicator_depends_update(self):
        # 自定义指标 依赖变量更新
        cdef double vol_inc
        if self.i == 0:
            pass
        else:
            vol_inc = self.vol[self.i] - self.vol[self.i-1]
            self.customized_indicator_max_part[self.i] = max(vol_inc, 0)
            self.customized_indicator_abs_part[self.i] = abs(vol_inc)
            if self.i == self.customized_indicator_n:
                self.customized_indicator_sma_max_part[self.i] = self.customized_indicator_max_part[self.i-self.customized_indicator_n+1:self.i+1].mean()
                self.customized_indicator_sma_abs_part[self.i] = self.customized_indicator_abs_part[self.i-self.customized_indicator_n+1:self.i+1].mean()
                self.customized_indicator[self.i] = self.customized_indicator_inc(self.customized_indicator_sma_max_part[self.i], self.customized_indicator_sma_abs_part[self.i])
            elif self.i > self.customized_indicator_n:
                self.customized_indicator_sma_max_part[self.i] = self.customized_indicator_sma_max_part[self.i-1] - self.customized_indicator_max_part[self.i-self.customized_indicator_n]/self.customized_indicator_n + self.customized_indicator_max_part[self.i]/self.customized_indicator_n
                self.customized_indicator_sma_abs_part[self.i] = self.customized_indicator_sma_abs_part[self.i-1] - self.customized_indicator_abs_part[self.i-self.customized_indicator_n]/self.customized_indicator_n + self.customized_indicator_abs_part[self.i]/self.customized_indicator_n
                self.customized_indicator[self.i] = self.customized_indicator_inc(self.customized_indicator_sma_max_part[self.i], self.customized_indicator_sma_abs_part[self.i])
          
    cdef cci_depends_update(self):
        # CCI 依赖变量更新
        # typical_price 已在mfi update 更新了
        if self.i >= self.cci_n-1:
            if self.i == self.cci_n-1:
                self.avg_typical_price[self.i] = self.typical_price[self.i-self.cci_n+1:self.i+1].mean()
            elif self.i > self.cci_n-1:
                self.avg_typical_price[self.i] = self.avg_typical_price[self.i-1] - self.typical_price[self.i-self.cci_n]/self.cci_n + self.typical_price[self.i]/self.cci_n
            self.mean_deviation[self.i] = abs(self.typical_price[self.i-self.cci_n+1:self.i+1] - self.avg_typical_price[self.i]).mean()
            self.cci[self.i] = self.CCI_inc(self.typical_price[self.i], self.avg_typical_price[self.i], self.mean_deviation[self.i])
            
    cdef kdj_depends_update(self):
        # KDJ 依赖变量更新
        if self.i >= self.kdj_n - 1:
            self.lowest_low[self.i] = self.low[self.i+1-self.kdj_n:self.i+1].min()
            self.highest_high[self.i] = self.high[self.i+1-self.kdj_n:self.i+1].max()
            self.rsv[self.i] = (self.close[self.i] - self.lowest_low[self.i])/(self.highest_high[self.i] - self.lowest_low[self.i]) * 100
            if self.i == self.kdj_n - 1 + self.k_n - 1:
                self.k[self.i] = self.rsv[self.i+1-self.k_n:self.i+1].mean()
            elif self.i > self.kdj_n - 1 + self.k_n - 1:
                self.k[self.i] = self.k[self.i-1] - self.rsv[self.i-self.k_n]/self.k_n + self.rsv[self.i]/self.k_n
                if self.i >= self.kdj_n - 1 + self.k_n - 1 + self.d_n - 1:
                    if self.i == self.kdj_n - 1 + self.k_n - 1 + self.d_n - 1:
                        self.d[self.i] = self.k[self.i+1-self.d_n:self.i+1].mean()
                    elif self.i > self.kdj_n - 1 + self.k_n - 1 + self.d_n - 1:
                        self.d[self.i] = self.d[self.i-1] - self.k[self.i-self.d_n]/self.d_n + self.k[self.i]/self.d_n
                    self.j[self.i] = 3*self.k[self.i] - 2*self.d[self.i]    
   
            
    cpdef new_bar(self, double new_high, double new_low, double new_close, double new_vol):
        self.i += 1
        self.high[self.i] = new_high
        self.low[self.i] = new_low
        self.close[self.i] = new_close
        self.vol[self.i] = new_vol
        

        self.dx_depends_update()
        self.mfi_depends_update()
        self.apo_depends_update()
        self.macd_depends_update()
        self.natr_depends_update()
        self.plus_dm_depends_update()    
        self.rsi_depends_update()
        self.tema_depends_update()
        self.customized_indicator_depends_update()
        self.cci_depends_update()
        self.kdj_depends_update()
        self.plus_di_depends_update()
        
    cpdef list get_indicators_name(self):
        return self.indicators_name
    
    cpdef np.ndarray get_indicators(self):
        return np.array([
                self.dx[self.i],
                self.adx[self.i],
                self.mfi[self.i],
                self.apo[self.i],
                self.macd[self.i],
                self.macdsignal[self.i],
                self.macdhist[self.i],
                self.natr[self.i],
                self.plus_dm[self.i],
                self.rsi[self.i],
                self.tema[self.i],
                self.customized_indicator[self.i],
                self.cci[self.i],
                self.rsv[self.i],
                self.k[self.i],
                self.d[self.i],
                self.j[self.i],
                self.natr_atr[self.i],
                self.plus_di[self.i]
        ]
        )

In [6]:
# test

import numpy as np 

size = 1000
high = np.random.random(size)
low = np.random.random(size)
close = np.random.random(size)
vol = np.random.random(size)


In [13]:
fast_n=12
slow_n=26
ta = TAHelper(mom_window=fast_n, std_window=slow_n)

DX = list()
ADX = list()
MFI = list()
APO = list()
MACD = list()
MACDsignal = list()
MACDhist = list()
NATR = list()
PLUS_DM = list()
RSI = list()
TEMA = list()
CUSTOMIZED_INDICATOR = list()
CCI = list()
RSV = list()
KDJ_K = list()
KDJ_D = list()
KDJ_J = list()
ATR = list()
PLUS_DI = list()

for i in range(len(high)):
    ta.new_bar(high[i], low[i], close[i], vol[i])
    indicators = ta.get_indicators()
    DX.append(indicators[0])
    ADX.append(indicators[1])
    MFI.append(indicators[2])
    APO.append(indicators[3])
    MACD.append(indicators[4])
    MACDsignal.append(indicators[5])
    MACDhist.append(indicators[6])
    NATR.append(indicators[7])
    PLUS_DM.append(indicators[8])
    RSI.append(indicators[9])
    TEMA.append(indicators[10])
    CUSTOMIZED_INDICATOR.append(indicators[11])
    CCI.append(indicators[12])
    RSV.append(indicators[13])
    KDJ_K.append(indicators[14])
    KDJ_D.append(indicators[15])
    KDJ_J.append(indicators[16])
    ATR.append(indicators[17])
    PLUS_DI.append(indicators[18])
    

In [9]:
import talib 
np.set_printoptions(threshold=200)

np.array(DX) - talib.DX(high, low, close, timeperiod=fast_n)

array([            nan,             nan,             nan, ...,
       -1.33226763e-14, -7.10542736e-15,  0.00000000e+00])

In [6]:
np.array(ADX) - talib.ADX(high, low, close, timeperiod=fast_n)

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [7]:
np.array(MFI) - talib.MFI(high, low, close, vol, timeperiod=fast_n)

array([           nan,            nan,            nan, ...,
       2.84217094e-14, 2.84217094e-14, 1.42108547e-14])

In [8]:
np.array(APO) - talib.APO(close, fastperiod=fast_n, slowperiod=slow_n)

array([            nan,             nan,             nan, ...,
       -2.22044605e-16, -1.66533454e-16, -1.66533454e-16])

In [9]:
macd1 = np.array(MACD)
signal1 = np.array(MACDsignal)
hist1 = np.array(MACDhist)
macd2, signal2, hist2 = talib.MACD(close, fastperiod=fast_n, slowperiod=slow_n)

In [10]:
macd1 - macd2

array([            nan,             nan,             nan, ...,
       -1.11022302e-16, -1.11022302e-16, -1.11022302e-16])

In [11]:
signal1 - signal2

array([            nan,             nan,             nan, ...,
       -7.97972799e-17, -8.67361738e-17, -9.71445147e-17])

In [12]:
hist1 - hist2

array([            nan,             nan,             nan, ...,
       -3.12250226e-17, -2.42861287e-17, -1.38777878e-17])

In [13]:
np.set_printoptions(threshold=200)
np.array(NATR) - talib.NATR(high, low, close, timeperiod=fast_n)

array([           nan,            nan,            nan, ...,
       2.84217094e-14, 0.00000000e+00, 1.42108547e-14])

In [14]:
np.set_printoptions(threshold=200)

np.array(PLUS_DM) - talib.PLUS_DM(high, low, timeperiod=fast_n)

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [19]:
np.set_printoptions(threshold=200)

np.array(RSI) - talib.RSI(close, timeperiod=fast_n)

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [16]:
np.array(TEMA) - talib.TEMA(close, timeperiod=slow_n)

array([           nan,            nan,            nan, ...,
       7.77156117e-16, 3.33066907e-16, 2.22044605e-16])

In [28]:
np.array(RSI_VOL) - talib.RSI(vol, timeperiod=fast_n)

array([            nan,             nan,             nan, ...,
       -7.10542736e-15,  0.00000000e+00,  0.00000000e+00])

In [33]:
np.array(CCI) - talib.CCI(high, low, close, timeperiod=fast_n)

array([           nan,            nan,            nan, ...,
       0.00000000e+00, 0.00000000e+00, 7.10542736e-15])

In [10]:
np.array(ATR) - talib.ATR(high, low, close, timeperiod=fast_n)

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [14]:
np.array(PLUS_DI) - talib.PLUS_DI(high, low, close, timeperiod=fast_n)

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [15]:
ta.get_indicators_name()

['DX',
 'ADX',
 'MFI',
 'APO',
 'MACD',
 'MACDsignal',
 'MACDhist',
 'NATR',
 'PLUS_DM',
 'RSI',
 'TEMA',
 'CUSTOMIZED_INDICATOR',
 'CCI',
 'RSV',
 'KDJ_K',
 'KDJ_D',
 'KDJ_J',
 'ATR',
 'PLUS_DI']